<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
* `DB_HOST`, `DB_NAME`, `DB_PASSWORD`, `DB_USER`
* `OGC_EDC_URL`
* `REFERENCE_DATA`


In [2]:
from edc import check_compatibility
check_compatibility("user-0.19.3")

This notebook is compatible with this base image version (user-0.19.3).

# DAPA Tutorial #2: Area - Sentinel-2

## Install dependencies and load libraries

Load Python libraries used in this tutorial.

In [3]:
import xarray as xr
import pandas as pd
import requests
import matplotlib
from ipyleaflet import Map, Rectangle, DrawControl, basemaps, basemap_to_tiles
%matplotlib inline

## Set DAPA endpoint

In [4]:
service_url = 'PLEASE INSERT YOU DAPA ENDPOINT HERE'
dapa_url = None

if not service_url.startswith('https://'):
    print('!!! ERROR: Please set your DAPA endpoint in the service_url variable !!!')
else:
    dapa_url = '{}/{}'.format(service_url, 'dapa')
    print(dapa_url)

!!! ERROR: Please set your DAPA endpoint in the service_url variable !!!


## Get collections supported by this endpoint
This request provides a list of collections. The path of each collection is used as starting path of this service.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
print(dapa_url)
collections = requests.get(dapa_url)
collections.json()

None


MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

## Get fields of collection Sentinel-2 L2A
The fields (or variables in other DAPA endpoints) can be retrieved in all requests to the DAPA endpoint. In addition to the fixed set of fields, "virtual" fields can be used to conduct math operations (e.g., the calculation of indices). 

In [ ]:
collection = 'S2L2A'

In [ ]:
fields_url = '{}/{}/{}'.format(dapa_url, collection, 'dapa/fields')
fields = requests.get(fields_url)
fields.json()

## Retrieve data as as raster aggregated by time

In [ ]:
url = '{}/{}/{}'.format(dapa_url, collection, 'dapa/area')
params = {
    'bbox': '11.49,48.05,11.66,48.22',
    'time': '2018-05-07T10:00:00Z/2018-05-07T12:00:00Z',
    'fields': 'NDVI=(B08-B04)/(B08%2BB04),NDBI=(B11-B08)/(B11%2BB08)',
    'aggregate': 'avg'
}

In [ ]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.14, 11.56),
    zoom=11
)

bbox = [float(coord) for coord in params['bbox'].split(',')]
rectangle = Rectangle(bounds=((bbox[1], bbox[0]), (bbox[3], bbox[2])))
m.add_layer(rectangle)
m

### Build request URL and conduct request

In [ ]:
params_str = "&".join("%s=%s" % (k, v) for k,v in params.items())

r = requests.get(url, params=params_str)
print(r.url)
print('Status code: {}'.format(r.status_code))

### Write raster dataset to GeoTIFF file

In [ ]:
with open('area_avg.tif', 'wb') as filew:
    filew.write(r.content)

### Open raster dataset with xarray

In [ ]:
ds = xr.open_rasterio('area_avg.tif')
ds

### Plot NDVI image

In [ ]:
ds[0].plot(cmap="RdYlGn")

## Output gdalinfo

In [ ]:
!gdalinfo -stats area_avg.tif